#### Load packages

In [10]:
import keras

In [11]:
from keras.models import Sequential

In [23]:
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout
import tensorflow as tf 

In [13]:
from matplotlib import pyplot as plt 

In [14]:
from keras.datasets import cifar100

In [35]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [36]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


#### Scaling the 8-bit values between 0 to 1

In [37]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#### One hot encoding for multi-class 


In [38]:
num_classes = 100

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [39]:
def VGG16(num_classes):
    model = Sequential([
        # Block 1
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        # Block 2
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        # Block 3
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        # Block 4
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        # Block 5
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        # Classification block
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model


In [40]:
num_classes = 100
vgg16_model = VGG16(num_classes)

In [41]:
vgg16_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 conv2d_40 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 conv2d_42 (Conv2D)          (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 8, 8, 128)        0         
 g2D)                                                 

In [42]:
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [43]:
run_hist = vgg16_model.fit(x_train, y_train,
              epochs=5,
              validation_data=(x_test, y_test),
              shuffle=True)

Epoch 1/5
1563/1563 [==============================] - 2568s 2s/step - loss: 4.6065 - accuracy: 0.0095 - val_loss: 4.6052 - val_accuracy: 0.0100
Epoch 2/5
1563/1563 [==============================] - 2530s 2s/step - loss: 4.6060 - accuracy: 0.0090 - val_loss: 4.6052 - val_accuracy: 0.0100
Epoch 3/5
1563/1563 [==============================] - 1812s 1s/step - loss: 4.6059 - accuracy: 0.0081 - val_loss: 4.6052 - val_accuracy: 0.0100
Epoch 4/5
1563/1563 [==============================] - 1395s 893ms/step - loss: 4.6059 - accuracy: 0.0089 - val_loss: 4.6052 - val_accuracy: 0.0100
Epoch 5/5
1563/1563 [==============================] - 1397s 894ms/step - loss: 4.6059 - accuracy: 0.0083 - val_loss: 4.6052 - val_accuracy: 0.0100


In [44]:
test_loss, test_acc = vgg16_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 71s 226ms/step - loss: 4.6052 - accuracy: 0.0100
Test accuracy: 0.009999999776482582


In [ ]:
fig, ax = plt.subplots()
ax.plot(run_hist.history["loss"],'r', marker='.', label="Train Loss")
ax.plot(run_hist.history["val_loss"],'b', marker='.', label="Validation Loss")
ax.legend()